In [0]:
%python
# Lista Diretórios
dbutils.fs.ls('/mnt')

In [0]:
%python
#Cria Diretório
dbutils.fs.mkdirs('/mnt/dados')

In [0]:
%python
# Lista Diretórios
dbutils.fs.ls('/mnt')

In [0]:
%python
# desmontando um diretório
#dbutils.fs.unmount("/mnt/<mount-name>")

In [0]:
%python

# código em python 

# configs = {"fs.azure.account.auth.type": "OAuth",
#           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
#           "fs.azure.account.oauth2.client.id": "<application-id>",
#           "fs.azure.account.oauth2.client.secret": "<secret-id>",
#           "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/<directory-id>/oauth2/token"}

# dbutils.fs.mount(
#   source = "abfss://<container-name>@<storage-account-name>.dfs.core.windows.net/",
#   mount_point = "/mnt/<mount-name>",
#   extra_configs = configs)


In [0]:
// val configs = Map(
//   "fs.azure.account.auth.type" -> "OAuth",
//   "fs.azure.account.oauth.provider.type" -> "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
//   "fs.azure.account.oauth2.client.id" -> "<application-id>",
//   "fs.azure.account.oauth2.client.secret" -> "<secret-id>",
//   "fs.azure.account.oauth2.client.endpoint" -> "https://login.microsoftonline.com/<directory-id>/oauth2/token")


// dbutils.fs.mount(
//   source = "abfss://<container-name>@<storage-account-name>.dfs.core.windows.net/",
//   mountPoint = "/mnt/dados",
//   extraConfigs = configs)

In [0]:
%python
dbutils.fs.ls("/mnt/dados")

In [0]:
dbutils.fs.ls("/mnt/dados/inbound")

In [0]:
%python
dbutils.fs.ls("/mnt/dados/inbound")

In [0]:
val path="dbfs:/mnt/dados/inbound/source-4-ds-train.json"
val df = spark.read.text(path)

In [0]:
display (df)

In [0]:
import org.apache.spark.sql.functions.get_json_object

In [0]:
df.select(get_json_object($"value","$.id") as "id").show()


In [0]:
val df_bronze = df.withColumn("id",get_json_object($"value","$.id"))

display(df_bronze)

In [0]:
import org.apache.spark.sql.functions.to_timestamp

In [0]:
val df_bronze = df.withColumn("id",get_json_object($"value","$.id"))
                  .withColumn("createdAt", to_timestamp(get_json_object($"value","$.createdAt")))
                  .withColumn("updatedAt", to_timestamp(get_json_object($"value","$.updatedAt")))

display(df_bronze)

In [0]:
val dbfs_path = "dbfs:/mnt/dados/bronze/zap_imoveis"
df_bronze.write.format("delta").mode(SaveMode.Overwrite).save(dbfs_path)